In [1]:
import pandas as pd

from serlo.mysql import db

df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()

/home/kulla/.local/share/asdf/installs/python/3.10.1/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,id,username,description,date,edits,reviews,comments,taxonomy
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,8.0,17.0
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0


In [6]:
df["date"] = df["date"].map(pd.to_datetime)
df2 = df[
    (df["edits"] + df["reviews"] + df["taxonomy"] == 0) &
    (df["description"] != "NULL") & (df["description"] != "") &
    (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
df2.sort_values(["date"], inplace=True)
df2

,id,username,description,date,edits,reviews,comments,taxonomy
2247,47562,Yoyo,"[[{""col"":9,""content"":""TU MÃÂ¼nchen""},{""col"":1...",2015-10-03 18:15:33,0.0,0.0,0.0,0.0
2399,49652,fabianseger,"[[{""col"":9,""content"":""![](/attachment/file/496...",2015-12-03 21:34:22,0.0,0.0,0.0,0.0
2631,53347,rudi,"[[{""col"":12,""content"":""Lehrer am Gymnasium Ste...",2016-03-09 16:46:52,0.0,0.0,0.0,0.0
3029,63395,Erenor,"[[{""col"":6,""content"":""Hallo Leute =)\n\nAndrea...",2016-09-17 07:58:32,0.0,0.0,0.0,0.0
3150,64803,T-Rom_or,"[[{""col"":24,""content"":""""}]]",2016-10-26 15:29:54,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
15802,219197,saulortiz,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-07-09 05:44:26,0.0,0.0,0.0,0.0
16098,223114,lucaswilliam1913,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-08-20 19:14:52,0.0,0.0,0.0,0.0
16313,224893,laimiro701,"{""plugin"":""rows"",""state"":[]}",2021-09-17 21:16:42,0.0,0.0,0.0,0.0
16474,226367,imperioninfomedia10,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2021-10-05 08:18:05,0.0,0.0,0.0,0.0


In [7]:
df2.to_csv("/tmp/users-to-delete.csv")